In [ ]:
!pip install google-play-scraper

In [ ]:
from google_play_scraper import app
import pandas as pd
import numpy as np

In [ ]:
from google_play_scraper import Sort, reviews

result, continuation_token = reviews(
    'com.', # Use apps id to scrape target apps
    lang='id', # Use your language
    country='id', # Use your country
    sort=Sort.MOST_RELEVANT,
    count=1000, # You can modify this with the amount of data you want to retrieve
    filter_score_with=None # You can use to filter with rating 1,2,3,4 or 5 or use None to get all
)

In [ ]:
df_busu = pd.DataFrame(np.array(result), columns=['review'])
df_busu = df_busu.join(pd.DataFrame(df_busu.pop('review').tolist()))
df_busu.head()

In [ ]:
len(df_busu.index)

In [ ]:
df_busu[['userName', 'score', 'at', 'content']].head()

In [ ]:
new_df = df_busu[['userName', 'score', 'at', 'content']]
sorted_df = new_df.sort_values(by='at', ascending=False)
sorted_df.head()

In [ ]:
my_df = sorted_df[['userName', 'score', 'at', 'content']]

In [ ]:
my_df = my_df[['content', 'score']]

In [ ]:
my_df.head()

In [ ]:
def labelling(score):
  if score < 3:
    return 'Negative'
  elif score == 4:
    return 'Positive'
  elif score == 5:
    return 'Positive'

my_df['Label'] = my_df['score'].apply(labelling)
my_df.head(50)

In [ ]:
my_df.to_csv('scrapped_data.csv', index=False)

In [ ]:
pd.set_option('display.max_column', None)
my_df = pd.read_csv('/content/scrapped_data.csv')
my_df.head(50)

In [ ]:
my_df.info()

In [ ]:
my_df.isna()

In [ ]:
my_df.isna().any()

In [ ]:
my_df.describe()

In [ ]:
my_df.isnull().sum()

In [ ]:
my_df.dropna(subset=['Label'], inplace = True)

In [ ]:
my_df.isnull().sum()

In [ ]:
my_df.head(50)

In [ ]:
my_df.to_csv('name-apps.csv', index=False)

#Text Pre-Processing

In [ ]:
df = pd.read_csv('/content/name-apps.csv')
df.head(50)

#Case Folding

In [ ]:
import re
def clean_text(df, text_field, new_text_field_name):
  my_df[new_text_field_name] = my_df[text_field].str.lower()
  my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))

  my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
  return my_df

In [ ]:
my_df['text_clean'] = my_df['content'].str.lower()
my_df['text_clean']
data_clean = clean_text(my_df, 'content', 'text_clean')
data_clean.head(10)

#Stopword Removal

In [ ]:
import nltk.corpus
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('indonesian') # you can modify this to your language

data_clean['text_Stopword'] = data_clean['text_clean'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
data_clean.head(50)

#Tokenizing

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
data_clean['text_tokens'] = data_clean['text_Stopword'].apply(lambda x: word_tokenize(x))
data_clean.head()

#Stemming (for Indonesian language)

In [ ]:
!pip install Sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}
hitung = 0

for document in data_clean['text_tokens']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
  term_dict[term] = stemmed_wrapper(term)
  hitung += 1
  print(hitung, ":", term, ":", term_dict[term])

print(term_dict)
print("------------------------")

def get_stemmed_term(document):
  return [term_dict[term] for term in document]

data_clean['text_steamindo'] = data_clean['text_tokens'].apply(lambda x:' '.join(get_stemmed_term(x)))
data_clean.head(20)

In [ ]:
data_clean.to_csv('result_TextPreProcessing_nameapp.csv', index=False)

#TF-IDF

In [ ]:
!pip install sklearn

In [ ]:
def praproses(text):
  text.re.sub('<[^>]*>', '', text)
  emoticons = re.findall('(?::|;|=)()(?:-)?(?:\)\(|D|P)', text)
  text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))

  return text

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_clean['content'], data_clean['Label'], test_size=0.20, random_state=0)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X_train)

In [ ]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(tfidf_train, y_train)

In [ ]:
X_train.toarray()

In [ ]:
y_pred = nb.predict(tfidf_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

clf = MultinomialNB()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)

print("MultinomialNB Accuracy:", accuracy_score(y_test, predicted))
print("MultinomialNB Precission:", precision_score(y_test, predicted, average="binary", pos_label="Negatif"))
print("MultinomialNB Recall:", recall_score(y_test, predicted, average="binary", pos_label="Negatif"))
print("MultinomialNB f1_score:", f1_score(y_test, predicted, average="binary", pos_label="Negatif"))

print(f'confussion_matrix:\n {confusion_matrix(y_test, predicted)}')
print('=====================================================\n')
print(classification_report(y_test, predicted, zero_division=0))

data_clean = pd.read_csv('result_TextPreProcessing_nameapp.csv')
